In [ ]:
###Models to do

In [1]:
import re
import xgboost as xgb
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)
#import torch.nn as nn
#import touch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random import randint
from Gather_AFL_Data import gatherer as gad
#from fdnn import feature_extractor as fex
import skopt
from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score, KFold
from joblib import dump, load
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBClassifier
from numpy import sort
import warnings
warnings.filterwarnings('ignore')

/home/chris/anaconda3/lib/python3.8/site-packages/xgboost/compat.py:93: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


2.2.0
2.4.3


In [2]:
import re
import xgboost as xgb
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)
#import torch.nn as nn
#import touch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random import randint
from Gather_AFL_Data import gatherer as gad
#from fdnn import feature_extractor as fex
import skopt
from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold
import pickle
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Activation
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score, KFold

2.2.0
2.4.3


In [3]:
#get headers
#feed this into a bigger function which specifies the amount of games to go through 
def get_headers(n_games):
    headers = ['Round', 'Home_Team', 'Away_Team', 'Venue', 'H_PAV_Sum', 'A_PAV_Sum']
    example_file = pd.read_csv('Data/Fremantle_clean_stats.csv')
    cl_h = example_file.columns
    cl_h = cl_h[:-5]
    ladder_header = ['Ladder Pos_H', 'Form_H', 'Season Wins_H', 'Season Loss_H', 'Season Draw_H']
    headers = [*headers, *ladder_header]
    j = 1
    while j <= n_games:
        for x in cl_h:
            if 'Match_ID' in x or 'Year' in x:
                continue
            x = 'H_'+ x + ' n-' + str(j)
            headers.append(x)
        j = j + 1
    j = 1
    ladder_header = ['Ladder Pos_A', 'Form_A', 'Season Wins_A', 'Season Loss_A', 'Season Draw_A']
    headers = [*headers, *ladder_header]
    while j <= n_games:
        for x in cl_h:
            if 'Match_ID' in x or 'Year' in x:
                continue
            x = 'A_'+ x + ' n-' + str(j)
            headers.append(x)
        j = j + 1
    return headers

In [4]:
def clean_headers(h):
    headers = []
    for x in h:
        if '<' in x or '>' in x:
            x = x.replace('<',"lt_")
            x = x.replace('>', "gt_")
            #print(x)
        headers.append(str(x))
    return headers

In [5]:
def generate_categorical_headers(h):
    cat_var = ['Round', 'Home_Team', 'Away_Team']
    skip = 0
    for x in h:
        if 'Round' in x:
            if (skip == 0):
                skip = 1
                continue
            cat_var.append(x)
            #print(x)
        elif 'Team_against_ID' in x:
            #print(x)
            cat_var.append(x)
        elif 'Venue' in x:
            cat_var.append(x)
    return cat_var

In [6]:
#one hot encode data and transform the X_data
#first redo, find the categorial variables
def ohe_data(x_data, enc, flag,cat_var):
    #data has not been previously one hot encoded
    if (flag == 0):
        #get columns with categorical data and drop from main DF
        categorical_data = x_data[cat_var]
        x_data = x_data.drop(cat_var, axis = 1)
        #define and fit new OHE. Use it on our categorical data by transforming
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        ohe = ohe.fit(categorical_data)
        categorical_data = ohe.transform(categorical_data)
        #get feature names for better labelling
        fn = ohe.get_feature_names(cat_var)
        #make a dataframe with new OHE data and feature names
            #would have been good to have coded it like this for my Masters project...
        categorical_data = pd.DataFrame(categorical_data)
        categorical_data.columns = fn
        #ensure that it won't get cranky about any different indexes(shouldn't be any but just a good check)
        x_data.reset_index(drop=True, inplace=True)
        categorical_data.reset_index(drop=True, inplace=True)
        #concatenate along column axis
        x_data = pd.concat([x_data, categorical_data], axis = 1)
    else:
        #same as above except used already fitted ohe
        categorical_data = x_data[cat_var]
        x_data = x_data.drop(cat_var, axis = 1)
        categorical_data = enc.transform(categorical_data)
        fn = enc.get_feature_names(cat_var)
        categorical_data = pd.DataFrame(categorical_data)
        categorical_data.columns = fn
        x_data.reset_index(drop=True, inplace=True)
        categorical_data.reset_index(drop=True, inplace=True)
        x_data = pd.concat([x_data, categorical_data], axis = 1, ignore_index=True)
        ohe = enc
    return x_data, ohe

In [7]:
#search parameters for best XGB classifier or best XGB regressor

def param_search(x_data, y_label, class_reg):

    def on_step(optim_result):
        """
        Callback meant to view scores after
        each iteration while performing Bayesian
        Optimization in Skopt"""
        score = xgb_bayes_search.best_score_
        print("best score: %s" % score)
        if score >= 0.98:
            print('Interrupting!')
            return True
    X_train, X_test, y_train, y_test = train_test_split(x_data, y_label, test_size=0.1, random_state=157732)
    print("X_train shape: " + str(X_train.shape))
    print("X_test shape: " + str(X_test.shape))
  #  print("y_train shape: " + str(y_train.shape))
  #  print("y_test shape: " + str(y_test.shape))
    space ={'learning_rate': Real(0.01, 1.0, 'log-uniform'),
        'min_child_weight': Integer(0, 10),
        'max_depth': Integer(0, 50),
        'max_delta_step': Integer(0, 20),
        'subsample': Real(0.01, 1.0, 'uniform'),
        'colsample_bytree': Real(0.01, 1.0, 'uniform'),
        'colsample_bylevel': Real(0.01, 1.0, 'uniform'),
        'reg_lambda': Real(1e-9, 1000, 'log-uniform'),
        'reg_alpha': Real(1e-9, 1.0, 'log-uniform'),
        'gamma': Real(1e-9, 0.5, 'log-uniform'),
        'min_child_weight': Integer(0, 5),
        'n_estimators': Integer(50, 200),
        'scale_pos_weight': Real(1e-6, 500, 'log-uniform')}
    if(class_reg == 0):
        xgbclass = xgb.XGBClassifier(random_state=27022013)
    else:
        xgbclass = xgb.XGBRegressor(random_state=27022013)
    xgb_bayes_search = BayesSearchCV(xgbclass, space, n_iter=60, # specify how many iterations
                                    scoring=None, n_jobs=1, cv=10, verbose=1, random_state=42, n_points=12,
                                 refit=True)
  #  kk = np.isinf(X_train)
  #  if True in kk:
  #  	print("aaaaaaa")
  #  kk = np.isinf(y_train)
  #  if True in kk:
  #  	print("reeeeeee")
    try:
        xgb_bayes_search.fit(X_train, y_train.values.ravel(), callback = on_step)
    except:
        xgb_bayes_search.fit(X_train, y_train.values.ravel(), callback = on_step)
  #  print("BEST PARAMS ARE HERE")
 #   print(xgb_bayes_search.best_params_)
    model = xgb_bayes_search.best_estimator_

    return model

In [8]:
#x = all data for classifier
#y = label of whether home team or away team won
#m = xgb classifier for winner prediction
#mx = all data for regressor
#my = labels of margins
    #try absolute values
#mm = xgb regressor for margin prediction
def eval_xgb_games_margins(x, y, m, mx, my, mm):
    results = []
    error = []
    count = 0
    best_w = m
    high_w = 0
    best_m = mm
    high_m = 100
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state = 27022013)
    print(x)
    for train,test in cv.split(x,y):
       # print(len(train))
        count = count + 1
        print("Split: " + str(count))
        #comment out fit steps for random forest i guess lol
        prediction = m.fit(x.loc[train],y.loc[train]).predict_proba(x.loc[test])
        margin_pred = mm.fit(mx.loc[train], my.loc[train])
        print("variables for auroc curve done. Processing fold accuracy + checking best model")
        y_pred = m.predict(x.loc[test])
        #print(y_pred)
        m_pred = mm.predict(mx.loc[test])
        print(m_pred)
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y.loc[test], predictions)
        pcent = accuracy * 100.0
        print("The accuracy of this model is" + str(pcent))
        rmse = sqrt(mean_squared_error(m_pred, my.loc[test]))
        print("The rmse of this model is" + str(rmse))
        results.append(pcent)
        error.append(rmse)
        #change the best model to equal current model
        if(pcent > high_w):
            print("found new best classify")
            best_w = m
            high_w = pcent
        if(rmse < high_m):
            print("found best new margin")
            best_m = mm
            high_m = rmse
    print("Best win percentage split = " +str(high_w))
    print("Best margin rmse = "+str(high_m))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    print("Training Testing Margins: %.2f%% (%.2f%%)" % (np.mean(error), np.std(error)))
    return best_w, best_m

In [9]:
def eval_threshold_cv(x, y, m):
    results = []
    error = []
    count = 0
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state = 27022013)
    for train,test in cv.split(x,y):
       # print(len(train))
        count = count + 1
        #comment out fit steps for random forest i guess lol
        prediction = m.fit(x.loc[train],y.loc[train]).predict_proba(x.loc[test])
        y_pred = m.predict(x.loc[test])
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y.loc[test], predictions)
        pcent = accuracy * 100.0
        results.append(pcent)
    mean_acc = (np.mean(results))
    return mean_acc

In [10]:
def threshold_search(x_data, y_label, best_xgb_clas):
    thresholds = sort(best_xgb_clas.feature_importances_)
    threshold_array = []
    threshold_accuracies = []
    for thresh in thresholds:
        # select features using threshold
        selection = SelectFromModel(best_xgb_clas, threshold=thresh, prefit=True)
        selection_x_data = selection.transform(x_data)
        selection_x_data = pd.DataFrame(selection_x_data)
        if(thresh > 0 ):
            threshold_array.append(thresh)
            selection_model = XGBClassifier()
            acc = eval_threshold_cv(selection_x_data, y_label, selection_model)
            threshold_accuracies.append(acc)
            print("Thresh=%.10f, n=%d, Accuracy: %.2f%%" % (thresh, selection_x_data.shape[1], acc))
    i = 0
    max_acc = 0
    max_i = 0
    max_thresh = 0
    for x in threshold_array:
        current_acc = threshold_accuracies[i]
        if(current_acc > max_acc):
            max_acc = current_acc
            max_thresh = x
            max_i = i
        i = i + 1
    print("max accuracy is: " + str(max_acc) + "for threshold: " + str(max_thresh))
    return max_thresh

In [11]:
#make this a method param later
#n_games = 5
def run_all_models(n_games):
    #load headers and then subsequently categorical headers
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')

    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, na_enc, 0, cv)
    filename = 'Models/ohe_'+str(n_games)+'_no_2021_games.dat'
    pickle.dump(ohe, open(filename, "wb"))
    #reset headers
    feature_names = x_data.columns

    #loads the ylabel matrix,
    y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

    #loads margin as the y_label
    margin_label = pd.read_csv('Data/assembled_margin_ymatrix_no2020'+str(n_games)+'_games.csv')

    print(margin_label.shape)
    print(y_label.shape)
    print(x_data.shape)

    #regex solution which is apparently necessary??
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_data.columns.values]

    #optimise XGBoost model

    #for predicting win
    model = param_search(x_data, y_label, 0)
    #for predicting margin
    #margin_label = abs(margin_label)
    margin_model = param_search(x_data, margin_label, 1)
    #margin_model = pickle.load(open("Models/best_xgb_reg_no2020_"+str(n_games)+"_games.dat", "rb"))

   # margin_label = abs(margin_label)
    print(margin_label)
    best_xgb_clas, best_xgb_reg = eval_xgb_games_margins(x_data, y_label, model, x_data, margin_label, margin_model)

    #save
    pickle.dump(best_xgb_clas, open("Models/best_xgb_clas_no2020_"+str(n_games)+"_games.dat", "wb"))
    pickle.dump(best_xgb_reg, open("Models/best_xgb_reg_no2020_"+str(n_games)+"_games.dat", "wb"))
    
    best_threshold = threshold_search(x_data, y_label, best_xgb_clas)
    selection = SelectFromModel(best_xgb_clas, threshold=best_threshold, prefit=True)
    print(selection)
    selection_x_data = selection.transform(x_data)
    selection_x_data = pd.DataFrame(selection_x_data)

    fs_filename = 'Models/fs_criteria_'+str(n_games)+'.dat'
    pickle.dump(selection, open(fs_filename, "wb"))

    fs_model = param_search(selection_x_data, y_label, 0)

    best_xgb_fs_clas, best_xgb_reg = eval_xgb_games_margins(selection_x_data, y_label, fs_model, x_data, margin_label, margin_model)

    pickle.dump(best_xgb_fs_clas, open("Models/best_xgb_clas_FS_no2020_"+str(n_games)+"_games.dat", "wb"))

In [12]:
#as one big script to go through n_games
n_games = [2,3,10]
for n in n_games:
    run_all_models(n)

(1864, 1)
(1864, 1)
(1864, 733)
X_train shape: (1677, 733)
X_test shape: (187, 733)
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   55.8s finished


best score: 0.6595110316040549
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.6min finished


best score: 0.6595110316040549
Fitting 10 folds for each of 12 candidates, totalling 120 fits


...

best score: 0.6799007444168734
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.7min finished


best score: 0.6799007444168734
         0       1     2    3    4    5    6     7     8     9    ...  826  \
0     239.37  261.62   5.0  7.0  0.0  1.0  0.0 -44.0  64.0   7.0  ...  0.0   
1     178.78  239.86  18.0  0.0  0.0  1.0  0.0 -94.0  59.0  15.0  ...  0.0   
2     240.11  240.13   9.0  5.0  0.0  0.0  1.0  -7.0  78.0  16.0  ...  1.0   
3     175.47  262.28  13.0  4.0  0.0  1.0  0.0 -52.0  68.0  13.0  ...  0.0   
4     219.37  196.24  16.0  2.0  0.0  1.0  0.0 -68.0  54.0  24.0  ...  0.0   
...      ...     ...   ...  ...  ...  ...  ...   ...   ...   ...  ...  ...   
1787  200.00  207.40  16.0  0.0  0.0  1.0  0.0 -50.0  53.0  23.0  ...  0.0   
1788  220.40  179.32  14.0  0.0  0.0  1.0  0.0 -15.0  53.0  23.0  ...  0.0   
1789  263.09  191.80   2.0  1.0  0.0  1.0  1.0  49.0  84.0  21.0  ...  0.0   
1790  193.38  207.90   1.0  1.0  0.0  1.0  1.0  59.0  37.0  17.0  ...  0.0   
1791  153.62  178.66  11.0  0.0  0.0  1.0  0.0  -5.0  72.0  24.0  ...  0.0   

      827  828  829  830  831  8

variables for auroc curve done. Processing fold accuracy + checking best model
[ 2.14286327e+01 -1.70193958e+01  1.19417486e+01  6.85975313e-01
 -6.23287010e+00  1.77161562e+00 -2.80473042e+00  2.98520241e+01
  1.69651108e+01 -8.61943722e+00  1.91369057e+01  3.26652336e+01
  2.61650944e+01  1.58255348e+01  1.72720413e+01  1.17984648e+01
  1.00658798e+01  9.68656826e+00 -8.83624744e+00  3.23763351e+01
  1.27448349e+01 -3.22147293e+01  4.15957146e+01  4.12742538e+01
  3.33738861e+01  1.77099686e+01 -1.01546745e+01  2.05423031e+01
  1.30153694e+01  2.93505549e+00  4.77010679e+00  3.57023087e+01
 -5.72927666e+00 -3.20237427e+01  1.43888254e+01 -2.68427429e+01
 -1.08134975e+01  1.41286695e+00 -9.14008236e+00  1.36422653e+01
  1.66373501e+01  2.47058907e+01  1.42997341e+01 -1.54639158e+01
 -4.39305067e+00  4.81356525e+00 -2.46525741e+00 -8.38145065e+00
  2.65930996e+01 -2.58508091e+01  3.64270401e+01 -1.42754412e+00
  2.44473686e+01  9.13935852e+00 -4.83723593e+00  1.23150101e+01
  1.4541621

variables for auroc curve done. Processing fold accuracy + checking best model
[ 6.8014278e+00 -2.3345900e+01  1.9973986e+01 -2.1485550e+01
  1.5383403e+01  3.6917789e+01 -2.8767136e+01  1.3391185e+01
 -1.0002632e+01  1.9956408e+01  5.6401606e+00  8.6418753e+00
 -1.4251021e+00  4.5137703e+01  1.1715598e+01  7.2873950e+00
 -1.8350595e+01  4.0813293e+00  8.5418806e+00 -1.8557892e+01
 -2.6078947e+01  2.4349503e+01 -5.2763090e+00 -1.6620916e+01
  1.8644409e+01  1.3544348e+01 -1.3635923e+01  3.5394299e+01
  4.8465591e+01  2.2135275e+01 -2.9798458e+01 -1.3754137e+01
  2.3341301e+01  4.9669189e+01  1.7336901e+01  1.3807531e+01
  2.1828964e+01 -2.0597937e+00 -2.4869750e+00 -7.9649010e+00
 -3.1076090e+01  1.9296929e+00  9.2189903e+00  3.3266293e+01
 -1.6850362e+00  7.1939659e-01  3.9844048e+01  3.7980251e+01
  1.9584724e+01  1.2326142e+01  1.0171807e+01 -1.7200446e+00
 -7.5775928e+00 -1.0612187e+01  1.2292419e+01 -2.5785555e+01
  1.2481314e+01  2.4127344e+01  1.4692000e+01 -2.0409739e+01
  3.69

# Best of 3 Models

In [ ]:
def prep_data_test(n_games,ohe):
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')
    #manual update cbf automating this part as this is just a rough check to see how well best of 3 does
    x_data = x_data.tail(1774)
    
    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, ohe, 1, cv)
    
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_data.columns.values]

    return x_data

In [ ]:
def load_test_models(n):
    clf = pickle.load(open("Models/best_xgb_clas_no2020_"+str(n)+"_games.dat", "rb"))
    ohe = pickle.load(open("Models/ohe_"+str(n)+"_no_2021_games.dat", "rb"))
    return clf, ohe

In [ ]:
#load data
y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no202010_games.csv')

model_2, ohe = load_test_models(2)
x_2 = prep_data_test(2,ohe)
model_3, ohe = load_test_models(3)
x_3 = prep_data_test(3, ohe)
model_10, ohe = load_test_models(10)
x_10 = prep_data_test(10, ohe)


In [ ]:
results = []
count = 0
high_w = 0
cv = StratifiedKFold(n_splits=10, shuffle=True)
y = y_label
for train,test in cv.split(x_2,y):
    count = count + 1
    print("Split: " + str(count))
    prediction = model_2.fit(x_2.loc[train],y.loc[train]).predict_proba(x_2.loc[test])
    prediction = model_3.fit(x_3.loc[train],y.loc[train]).predict_proba(x_3.loc[test])
    prediction = model_10.fit(x_10.loc[train],y.loc[train]).predict_proba(x_10.loc[test])
    print("variables for auroc curve done. Processing fold accuracy + checking best model")
    d = {'y_pred_2':model_2.predict(x_2.loc[test]), 'y_pred_3':model_3.predict(x_3.loc[test]),
         'y_pred_10':model_10.predict(x_10.loc[test])}
    pred_df = pd.DataFrame(data=d)
    summary_ave_data = pred_df.copy()
    summary_ave_data['average'] = summary_ave_data.mean(numeric_only=True, axis=1)
    print(summary_ave_data)
    y_pred = summary_ave_data["average"].tolist()
    print(y_pred)
    predictions = [round(value) for value in y_pred]
    print(predictions)
    #sees how accurate the model was when testing the test set
    accuracy = accuracy_score(y.loc[test], predictions)
    pcent = accuracy * 100.0
    print("The accuracy of this model is" + str(pcent))
    results.append(pcent)
print("Best win percentage split = " +str(high_w))
print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))

# Reduced Multicollinearity Models

In [ ]:
from numpy import loadtxt
from numpy import sort
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
import warnings
warnings.filterwarnings('ignore')
n_games = 2
model = pickle.load(open("Models/best_xgb_clas_no2020_"+str(n_games)+"_games.dat", "rb"))
thresholds = sort(model.feature_importances_)
print(thresholds)

In [ ]:

#find multicollinearaity and then reduce it

#def feature_select(x_data, clf):
n_games = 2
h = get_headers(n_games)
h = clean_headers(h)
cv = generate_categorical_headers(h)

#to get names of teams from index
g = gad()
teams = g.createTeamDict()

#load Data
x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')

#make empty OHE object
na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

#one hot encode data with new one hot encoder, saves ohe for later use
x_data, ohe = ohe_data(x_data, na_enc, 0, cv)
#filename = 'Models/ohe_'+str(n_games)+'_no_2021_games_fs.dat'
#pickle.dump(ohe, open(filename, "wb"))
#reset headers
feature_names = x_data.columns

#loads the ylabel matrix,
y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

#loads margin as the y_label
margin_label = pd.read_csv('Data/assembled_margin_ymatrix_no2020'+str(n_games)+'_games.csv')

print(margin_label.shape)
print(y_label.shape)
print(x_data.shape)

#regex solution which is apparently necessary??
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
x_data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_data.columns.values]

model = pickle.load(open("Models/best_xgb_clas_no2020_"+str(n_games)+"_games.dat", "rb"))
margin_model = pickle.load(open("Models/best_xgb_reg_no2020_"+str(n_games)+"_games.dat", "rb"))

# margin_label = abs(margin_label)
print(margin_label)
best_xgb_clas, best_xgb_reg = eval_xgb_games_margins(x_data, y_label, model, x_data, margin_label, margin_model)

best_threshold = threshold_search(x_data, y_label, best_xgb_clas)
selection = SelectFromModel(best_xgb_clas, threshold=best_threshold, prefit=True)
print(selection)
selection_x_data = selection.transform(x_data)
selection_x_data = pd.DataFrame(selection_x_data)

fs_filename = 'Models/fs_criteria_'+str(n_games)+'.dat'
pickle.dump(selection, open(fs_filename, "wb"))

fs_model = param_search(selection_x_data, y_label, 0)

best_xgb_fs_clas, best_xgb_reg = eval_xgb_games_margins(selection_x_data, y_label, fs_model, x_data, margin_label, margin_model)

pickle.dump(best_xgb_fs_clas, open("Models/best_xgb_clas_FS_no2020_"+str(n_games)+"_games.dat", "wb"))
    #return x_data

In [ ]:
def eval_threshold_cv(x, y, m):
    results = []
    error = []
    count = 0
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state = 27022013)
    for train,test in cv.split(x,y):
       # print(len(train))
        count = count + 1
        #comment out fit steps for random forest i guess lol
        prediction = m.fit(x.loc[train],y.loc[train]).predict_proba(x.loc[test])
        y_pred = m.predict(x.loc[test])
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y.loc[test], predictions)
        pcent = accuracy * 100.0
        results.append(pcent)
    mean_acc = (np.mean(results))
    return mean_acc

In [ ]:
def threshold_search(x_data, y_label, best_xgb_clas):
    thresholds = sort(best_xgb_clas.feature_importances_)
    threshold_array = []
    threshold_accuracies = []
    for thresh in thresholds:
        # select features using threshold
        selection = SelectFromModel(best_xgb_clas, threshold=thresh, prefit=True)
        selection_x_data = selection.transform(x_data)
        selection_x_data = pd.DataFrame(selection_x_data)
        if(thresh > 0 ):
            threshold_array.append(thresh)
            selection_model = XGBClassifier()
            acc = eval_threshold_cv(selection_x_data, y_label, selection_model)
            threshold_accuracies.append(acc)
            print("Thresh=%.10f, n=%d, Accuracy: %.2f%%" % (thresh, selection_x_data.shape[1], acc))
    i = 0
    max_acc = 0
    max_i = 0
    max_thresh = 0
    for x in threshold_array:
        current_acc = threshold_accuracies[i]
        if(current_acc > max_acc):
            max_acc = current_acc
            max_thresh = x
            max_i = i
        i = i + 1
    print("max accuracy is: " + str(max_acc) + "for threshold: " + str(max_thresh))
    return max_thresh

In [ ]:
n_2_thresh = 
n_3_thresh =
n_10_thresh = 

In [ ]:
#make this a method param later
#n_games = 5
def run_all_models(n_games):
    #load headers and then subsequently categorical headers
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')

    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, na_enc, 0, cv)
    filename = 'Models/ohe_'+str(n_games)+'_no_2021_games_fs.dat'
    pickle.dump(ohe, open(filename, "wb"))
    #reset headers
    feature_names = x_data.columns
    
    #DO STUFF Here
    x_data = reduce_MC(x_data)

    #loads the ylabel matrix,
    y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

    #loads margin as the y_label
    margin_label = pd.read_csv('Data/assembled_margin_ymatrix_no2020'+str(n_games)+'_games.csv')

    print(margin_label.shape)
    print(y_label.shape)
    print(x_data.shape)

    #regex solution which is apparently necessary??
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_data.columns.values]

    #optimise XGBoost model

    #for predicting win
    model = param_search(x_data, y_label, 0)
    #for predicting margin
    #margin_label = abs(margin_label)
    margin_model = clf = pickle.load(open("Models/best_xgb_reg_no2020_"+str(n)+"_games.dat", "rb"))

   # margin_label = abs(margin_label)
    print(margin_label)
    best_xgb_clas, best_xgb_reg = eval_xgb_games_margins(x_data, y_label, model, margin_label, margin_model)

    #save
    pickle.dump(best_xgb_clas, open("Models/best_xgb_clas_no2020_"+str(n_games)+"_games_fs.dat", "wb"))
    #pickle.dump(best_xgb_reg, open("Models/best_xgb_reg_no2020_"+str(n_games)+"_games.dat", "wb"))

# Clean Data Testing Ground

In [ ]:
def create_R_TeamDict():
    teams = {
    "1" : "Adelaide Crows",
    "2" : "Brisbane Lions",
    "3" : "Carlton",
    "4" : "Collingwood",
    "5" : "Essendon",
    "6" : "Fremantle",
    "7" : "Geelong Cats",
    "8" : "Gold Coast Suns",
    "9" : "GWS Giants",
    "10": "Hawthorn",
    "11": "Melbourne",
    "12": "North Melbourne",
    "13": "Port Adelaide",
    "14": "Richmond",
    "15": "St Kilda",
    "16": "Sydney Swans",
    "17": "West Coast Eagles",
    "18": "Western Bulldogs"
    }
    return teams

In [ ]:
team_int = 6

In [ ]:
pavs = pd.read_csv("R_Code/all_team_pavs.csv")
team_pavs = pavs.loc[(pavs["Team_ID"]==team_int)]
team_pavs

In [ ]:
df.shape

In [ ]:
def calc_sum_pav(year, rnd, team_int):
    #do calc
    g = gad()
    team_dict = g.createTeamDict()
    r_dict = create_R_TeamDict()
    current_team = (team_dict[str(team_int)])
    df = pd.read_csv("Data/"+current_team+'_clean_stats.csv')
    current_r_team = (r_dict[str(team_int)])
    lineups = pd.read_csv("R_Code/all_lineups.csv")
    lineups = lineups[lineups.isin([current_r_team]).any(axis=1)]
    lineups = lineups[lineups.isin([year]).any(axis=1)]
    lineups = lineups[lineups.isin([rnd]).any(axis=1)]
    lineups['team'] = team_int
    if(rnd < 11):
        lineups['year'] = (year-1)
    lineups.columns = ['year', 'teamname', 'roundNumber', 'firstname', 'surname', 'team']
    cols = ['team', 'year', 'firstname', 'surname']
    lineups = lineups[cols]
    all_pavs = pd.read_csv("R_Code/all_player_PAVs.csv")
    
    lineups.firstname = lineups.firstname.str.split(' ').str[0]
    all_pavs.firstname = all_pavs.firstname.str.replace(' ','')
    
    lineups = lineups.merge(all_pavs, how='inner', on=['team', 'year', 'firstname', 'surname'])
    if(lineups.shape[0] > 0):
        pav = lineups['PAV_total'].sum()
    else:
        pav = 999
    print(pav)
    pav_array = [year, rnd, team_int, pav]
    return pav_array

In [ ]:
pav_array = []
for team_id in range(1,19):
    for year in range(2022, 2023):
        for rnd in range(1,5):
            print(str(team_id) + '_' + str(year)+'_'+str(rnd))
            pa = calc_sum_pav(year, rnd, team_id)
            if(pa[3] == 999):
                continue
            pav_array.append(pa)
pav_df = pd.DataFrame(pav_array, columns=['Year', 'Round', 'Team_ID', 'Player_PAV_Total'])
all_pav_df = pd.read_csv('R_Code/all_team_pavs.csv')
all_pav_df = pd.concat([all_pav_df, pav_df], ignore_index=True)
print(pav_df)
print(all_pav_df)
all_pav_df.to_csv("R_Code/all_team_pavs.csv", header=True, index=False)

# Trend Based Dataset Testing

In [ ]:
#get assemble_df code from the clean_dataset csv
#do the usual stuff appended at the front eg. venue, ladder, round, H_team_id, A_team_id
#pick averageable statistics and get n_game average for home and away
#function to workout from clean_dataset home_team wins against away team in last 5
    #5-home_team_wins to get away team wins against oppo
    #append to new X_data
#save to new assemble_df.csv

match_id = 10576
team_id = 9
oppo_id = 6
g = gad()
teams = g.createTeamDict()
n_games = 3


In [ ]:
def find_form(ct, ot, match_id):
    g = gad()
    teams = g.createTeamDict()
    current_team = (teams[str(ct)])
    team_string = current_team+"_clean_stats.csv"
    df = pd.read_csv("Data/"+team_string)
    idx = df.index[df['Match_ID'] == match_id]
    my_idx = idx[0]
    df = df.loc[0:my_idx-1,:]
    prev_games = df.loc[(df['Team_against_ID']==ot)]
    #return last 5 games against each other and 8th column is whether the ct won against ot
    l = 5
    if(prev_games.shape[0] < 5):
        l = prev_games.shape[0]
    if(l==0):
        wins = 0
    else:
        prev_games = prev_games.iloc[-l: , 8]
        wins = prev_games.sum()
    return wins

In [ ]:
def create_prev_games(match_id, team_id, teams, n_games, oppo_id):
    margin = None
    ma = None
    y_label = None
    trend_cg = []
    current_team = (teams[str(team_id)])
    print(current_team)
    print(match_id)
    team_string = current_team+"_clean_stats.csv"
    df = pd.read_csv("Data/"+team_string)
    #drops ladder stats
    #finds where in the dataframe the current match is
    idx = df.index[df['Match_ID'] == match_id]
    #print(idx)
    my_idx = idx[0]
    #splits dataframe into game data and end of round ladder data
    l_df = df.iloc[:,-5:]
    t_df = df.iloc[: , :-5]
    current_year = t_df.loc[my_idx][1]
    if(current_year == 2020.0):
        print('game in 2020')
        margin = 8888
    else:
        #turns the WWWLL form column into # of W
        n_form = []
        for x in l_df['form']:
            if(len(x)<n_games):
                y=float(x.count("W"))
                n_form.append(y)
            else:
                x=x[-n_games:]
                y=float(x.count("W"))
                n_form.append(y)
        l_df['form'] = n_form
        #checks to make sure there is enough games to go through
        if(my_idx < (n_games)):
            print('Num of Prev Games Exceeds previous games')
            margin = 9999
        else:
            #start match array with the ladder values from end of previous round (as this would be current for predicting round)
            ma = l_df.loc[my_idx-1].values
            #finds both labels for models
            y_label = t_df.loc[my_idx]["H/A Win?"]
            margin = t_df.loc[my_idx]["Margin"]
            #start from the previous game to current game
            #i is to know how many games included
            i = 1
            #j finds the previous game and allows for 2020 exclusion
            j = 1
            while i <= n_games:
                year = t_df.loc[my_idx-j][1]
                if(year == 2020.0):
                    j = j + 1
                    continue
                cg = t_df.loc[my_idx-j][2:].values
                trend_cg.append(cg)
                i = i + 1
                j = j + 1
            my_df = pd.DataFrame(trend_cg, columns = t_df.columns[2:])
            my_df = my_df.iloc[:,3:-1]
            cg = my_df.mean(axis=0)
            ma = [*ma, *cg]
            h_form = find_form(team_id, oppo_id, match_id)
            o_form = find_form(oppo_id, team_id, match_id)
            form_diff = h_form-o_form
            ma.append(form_diff)
    return ma, y_label, margin

In [ ]:
home_array, y_label, margin = create_prev_games(5720, team_id, teams, 3, oppo_id)

In [ ]:
len(home_array)

In [ ]:
headers = ['Round', 'Home_Team', 'Away_Team', 'Venue']
example_file = pd.read_csv('Data/Fremantle_clean_stats.csv')
cl_h = example_file.columns
cl_h = cl_h[5:-6]
ladder_header = ['Ladder Pos_H', 'Form_H', 'Season Wins_H', 'Season Loss_H', 'Season Draw_H']
headers = [*headers, *ladder_header]
for x in cl_h:
    if 'Match_ID' in x or 'Year' in x:
        continue
    x = 'H_'+ x + ' avg'
    headers.append(x)
headers.append("H_Recent_Matchup_Diff")
ladder_header = ['Ladder Pos_A', 'Form_A', 'Season Wins_A', 'Season Loss_A', 'Season Draw_A']
headers = [*headers, *ladder_header]
for x in cl_h:
    if 'Match_ID' in x or 'Year' in x:
        continue
    x = 'A_'+ x + ' avg'
    headers.append(x)
headers.append("A_Recent_Matchup_Diff")
len(headers)

# Team Specific Models
## Averaged about 60%

In [ ]:
def get_team_specific_dataset(n_games, tn):
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()
    #load in x_data and y_label for n = 1 (quicker)
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')
    y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

    #finds team index
    team_idx = x_data.index[(x_data['Home_Team']==tn) | (x_data['Away_Team']==tn)]
    team_x_data = x_data.iloc[team_idx]
    team_y_data = y_label.iloc[team_idx]
    team_x_data.reset_index(drop=True, inplace=True)
    team_y_data.reset_index(drop=True, inplace=True)

    #quick shape check
    print(team_x_data.shape)
    print(team_y_data.shape)
    l = team_x_data.shape[0]

    #loop for new y_array
    i = 0
    new_team_y = []
    while i < l:
        x = team_x_data.iloc[i]
        y = team_y_data.iloc[i]
        #if our team for current model is the home team
        if(x["Home_Team"] == tn):
            #and the home team has won
            if(y["H/A Win?"] == 0):
                #set win val to 1
                w_val = 1
            #else they're the home team but the away team won
            else:
                #set win val to 0
                w_val = 0
        #else they're the away team
        else:
            #if the home team won, that means they lost
            if(y["H/A Win?"] == 0):
                w_val = 0
            #else they're the away team and they won
            else:
                w_val = 1
        i = i + 1
        new_team_y.append(w_val)
    ny = pd.DataFrame(new_team_y, columns=['Team Won?'])
    return team_x_data, ny

In [ ]:
x, y = get_team_specific_dataset(10, 1)

In [ ]:
def eval_team_xgb_games(x, y, m, n_games, tn):
    results = []
    error = []
    count = 0
    best_w = m
    high_w = 0
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    for train,test in cv.split(x,y):
       # print(len(train))
        count = count + 1
        print("Split: " + str(count))
        #comment out fit steps for random forest i guess lol
        prediction = m.fit(x.loc[train],y.loc[train]).predict_proba(x.loc[test])
        y_pred = m.predict(x.loc[test])
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y.loc[test], predictions)
        pcent = accuracy * 100.0
        results.append(pcent)
        #change the best model to equal current model
        if(pcent > high_w):
            best_w = m
            high_w = pcent
    print("Training Testing Accuracy for n_games="+str(n_games)+" team="+str(tn)+" : %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best_w, np.mean(results)

In [ ]:
def run_team_models(n_games, tn):
    #load headers and then subsequently categorical headers
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data, y_label = get_team_specific_dataset(n_games, tn)

    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, na_enc, 0, cv)
    filename = 'Models/Team_OHE/ohe_'+str(n_games)+"_team_specific_"+str(tn)+'_no_2020_games.dat'
    pickle.dump(ohe, open(filename, "wb"))
    #reset headers
    feature_names = x_data.columns

    #regex solution which is apparently necessary??
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_data.columns.values]

    #optimise XGBoost model
    #print(x_data)
    #for predicting win
    model = param_search(x_data, y_label, 0)
    best_xgb_clas, model_av = eval_team_xgb_games(x_data, y_label, model, n_games, tn)

    #save
    pickle.dump(best_xgb_clas, open("Models/Team_Models/best_xgb_clas_no2020_"+str(n_games)+"_team_specific_"+str(tn)+'_.dat', "wb"))
    return model_av

In [ ]:
import warnings
warnings.filterwarnings('ignore')
for n in range(1,11):
    averages = []
    for tn in range(1,19):
        a = run_team_models(n, tn)
        averages.append(a)
    print("Accuracy for Team Models for n_games= "+str(n)+" : %.2f%% (%.2f%%)" % (np.mean(averages), np.std(averages)))

# Nearest Centroid

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from Gather_AFL_Data import gatherer as gad
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
import pickle
from sklearn.neighbors import NearestCentroid
import re
from numpy import arange
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.naive_bayes import BernoulliNB

In [ ]:
def load_data(n_games):
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')

    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, na_enc, 0, cv)

    #reset headers
    feature_names = x_data.columns

    #loads the ylabel matrix,
    y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

    #loads margin as the y_label
    margin_label = pd.read_csv('Data/assembled_margin_ymatrix_no2020'+str(n_games)+'_games.csv')

    print(margin_label.shape)
    print(y_label.shape)
    print(x_data.shape)

    #regex solution which is apparently necessary??
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_data.columns.values]
    return x_data, y_label

In [ ]:
def params_NC(x_data, y_label):
# define model
    print("optimising hyperparameters")
    model = NearestCentroid()
    # define model evaluation method
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # define grid
    grid = dict()
    grid['shrink_threshold'] = arange(0, 1.01, 0.01)
    grid['metric'] = ['euclidean', 'manhattan']
    # define search
    search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=-1)
    # perform the search
    results = search.fit(x_data, y_label.values.ravel())
    # summarize
    print('Mean Accuracy: %.3f' % results.best_score_)
    print('Config: %s' % results.best_params_)
    return results.best_params_

In [ ]:
def eval_NC(x, y, m):
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    results = []
    print(x)
    count = 0
    best_w = m
    high_w = 0
    for train,test in cv.split(x,y):
        count = count + 1
        print("Split: " + str(count))
        x_train = x.loc[train]
        y_train = y.loc[train].values.ravel()
        x_test = x.loc[test]
        y_test = y.loc[test]
        m.fit(x_train,y_train)
        y_pred = m.predict(x_test)
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y_test, predictions)
        pcent = accuracy * 100.0
        results.append(pcent)
        if(pcent > high_w):
            print("found new best classify")
            best_w = m
            high_w = pcent
    print("Best win percentage split = " +str(high_w))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best_w

In [ ]:
def run_all_NC(n):
    x_data, y_label = load_data(n)
    best_params = params_NC(x_data, y_label)
    model = NearestCentroid(**best_params)
    best_NC_clas = eval_NC(x_data, y_label, model)
    pickle.dump(best_NC_clas, open("Models/best_NC_clas_no2020"+str(n_games)+'_games.sav', 'wb'))

In [ ]:
n_games = [1,2,3,4,5,6,7,10]
for n in n_games:
    run_all_NC(n)

# BernoulliNB

In [ ]:
def params_BNB(x_data, y_label):
# define model
    print("optimising hyperparameters")
    model = BernoulliNB()
    # define model evaluation method
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # define grid
    grid = dict()
    grid['alpha'] = arange(0.01, 1.01, 0.01)
    # define search
    search = GridSearchCV(model, grid, scoring='accuracy', cv=cv, n_jobs=1, verbose=1)
    # perform the search
    results = search.fit(x_data, y_label.values.ravel())
    # summarize
    print('Mean Accuracy: %.3f' % results.best_score_)
    print('Config: %s' % results.best_params_)
    return results.best_params_

In [ ]:
def eval_BNB(x, y, m):
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    results = []
    print(x)
    count = 0
    best_w = m
    high_w = 0
    for train,test in cv.split(x,y):
        count = count + 1
        print("Split: " + str(count))
        x_train = x.loc[train]
        y_train = y.loc[train].values.ravel()
        x_test = x.loc[test]
        y_test = y.loc[test]
        m.fit(x_train,y_train)
        y_pred = m.predict(x_test)
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y_test, predictions)
        pcent = accuracy * 100.0
        results.append(pcent)
        if(pcent > high_w):
            print("found new best classify")
            best_w = m
            high_w = pcent
    print("Best win percentage split = " +str(high_w))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best_w

In [ ]:
def run_all_BNB(n):
    x_data, y_label = load_data(n)
    best_params = params_BNB(x_data, y_label)
    model = BernoulliNB(**best_params)
    best_BNB_clas = eval_BNB(x_data, y_label, model)
    pickle.dump(best_BNB_clas, open("Models/best_BNB_clas_no2020"+str(n_games)+'_games.sav', 'wb'))

In [ ]:
n_games = [1,2,3,4,5,6,7,10]
for n in n_games:
    run_all_BNB(n)

# LAZY PREDICT

In [ ]:
import pandas as pd
import numpy as np
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.model_selection import train_test_split
from Gather_AFL_Data import gatherer as gad
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
import pickle

In [ ]:
def eval_lazy_predict(x, y, my):
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    all_clf_models = []
    all_reg_models = []
    count = 0
    for train,test in cv.split(x,y):
       # print(len(train))
        count = count + 1
        print("Split: " + str(count))
        #classifier
        X_train = x.loc[train]
        X_test = x.loc[test]
        y_train = y.loc[train]
        y_test = y.loc[test]
        my_train = my.loc[train]
        my_test = my.loc[test]
        clf = LazyClassifier(predictions=True)
        reg = LazyRegressor(predictions=True)
        if(count == 1):
            all_clf_models, predictions = clf.fit(X_train, X_test, y_train, y_test)
          #  all_reg_models, reg_predictions = reg.fit(X_train, X_test, my_train, my_test)
            all_clf_models.sort_index(inplace=True)
           # all_reg_models.sort_index(inplace=True)
            #print(all_models)
        else:
            new_clf_models, predictions = clf.fit(X_train, X_test, y_train, y_test)
           # new_reg_models, reg_predictions = reg.fit(X_train, X_test, my_train, my_test)
            new_clf_models.sort_index(inplace=True)
           # new_reg_models.sort_index(inplace=True)
            #print(new_models)
            all_clf_models = all_clf_models.add(new_clf_models)
           # all_reg_models = all_reg_models.add(new_reg_models)
    return all_clf_models#, all_reg_models

In [ ]:
def run_all_lp(n_games):
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')

    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, na_enc, 0, cv)

    #reset headers
    feature_names = x_data.columns

    #loads the ylabel matrix,
    y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

    #do margins in a second when this works
    margin_label = pd.read_csv('Data/assembled_margin_ymatrix_no2020'+str(n_games)+'_games.csv')
    margin_label = abs(margin_label)

    clf_lp = eval_lazy_predict(x_data, y_label, margin_label)

    clf_lp.sort_values(by='Accuracy', ascending=False, inplace=True)
    #reg_lp.sort_values(by='RMSE', inplace=True)
    pct_lp = clf_lp*10
    print(pct_lp)

In [ ]:
#as one big script to go through n_games
n_games = [1,2,3,4,5,6,7,10]
for n in n_games:
    run_all_lp(n)

# Deep Learningggg

In [ ]:
def build_DNN_model(x_len):
    model = Sequential()
    model.add(Dense(63, input_dim = x_len))
    model.add(Activation('relu'))
    model.add(Dropout(0.03))
    model.add(BatchNormalization())

    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.02))

    model.add(Dense(16))
    model.add(Activation('relu'))

    model.add(Dense(8))
    model.add(Activation('relu'))
    #add output layer
    model.add(Dense(1, activation='linear'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)

    model.compile(loss="mean_squared_error", optimizer=opt, metrics=[tf.keras.metrics.RootMeanSquaredError()])
    print(model.summary())
    return model

In [ ]:
def build_CNN_model(x_len):
    #del model
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=14,
                     input_shape=(x_len, 1)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(filters=16, kernel_size=10,
                     input_shape=(32, 1)))
    model.add(Activation('linear'))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=10, kernel_size=8,
                     input_shape=(16, 1)))
    model.add(Activation('linear'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(64, activation='linear'))
    model.add(Dense(32, activation='linear'))
    model.add(Dense(16, activation='linear'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='linear'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"


    model.compile(loss="mean_squared_error", optimizer=opt, metrics=[tf.keras.metrics.RootMeanSquaredError()])

    print(model.summary())
    return model

In [ ]:
#flag = 0 (DNN)
#flag = 1 (CNN)
def eval_dl(x,y,k,flag):
    cv = StratifiedKFold(n_splits=k,shuffle=True)
    best_model = []
    results = []
    highest = 0
    i = 1
    for train,test in cv.split(x,y):
        if(flag == 0):
            model = build_DNN_model(x[train].shape[1])
        if(flag == 1):
            x = x.reshape(x.shape[0], x.shape[1], 1)
            model = build_CNN_model(x[train].shape[1])
        bs = ((x[train].shape[0])/20)
        bs = round(bs)
        history = model.fit(x[train], y[train], validation_data=(x[test], y[test]), epochs = 50, batch_size=bs)
        _, accuracy = model.evaluate(x[test], y[test], batch_size=bs, verbose=0)
        accuracy = accuracy * 100
        print("accuracy for model " + str(i) + " is " + str(accuracy))
        if(accuracy > highest):
            highest = accuracy
            best_model = model
        results.append(accuracy)
        i = i + 1
    print("highest accuracy is: " + str(highest))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best_model